In [1]:
import tensorflow as tf
import numpy as np
import os
import pickle

In [2]:
with open("war_text.pickle", "rb") as file:
    war_text = pickle.load(file)

In [3]:
all_text = ''.join(war_text)

In [4]:
bul_symbols = "АБВГДЕЖЗИЙКЛМНОПРСТУФХЦЧШЩЬЪЮЯабвгдежзийклмнопрстуфхцчшщьъюя"

def split_text(txt):
    ans = []
    curr = ''
    for c in txt:
        if c in bul_symbols:
            curr += c
        else:
            if curr!='': ans.append(curr)
            ans.append(c)
            curr = ''
    if curr!='': ans.append(curr)
    return ans

splited_text = split_text(all_text)
splited_text[:10]

['Кримската', ' ', 'война', ' ', '(', 'октомври', ' ', '1', '8', '5']

In [5]:
id2word = sorted(set(splited_text))
word2id = {w:i for i,w in enumerate(id2word)}
vocab_size = len(id2word)
vocab_size

38912

In [6]:
id2word = np.array(id2word)

In [7]:
text_as_int = np.array([word2id[w] for w in splited_text])
text_as_int.shape

(834632,)

In [8]:
window_size = 8
def create_train_data(txt_int):
    x = []
    y = []
    for i in range(len(txt_int) - window_size + 1):
        x.append(txt_int[i:i+window_size-1])
        y.append(txt_int[i+window_size-1])
    return np.asarray(x), np.array(y)

train_x, train_y = create_train_data(text_as_int)
for i in range(4):
    print(id2word[train_x[i]], id2word[train_y[i]])

['Кримската' ' ' 'война' ' ' '(' 'октомври' ' '] 1
[' ' 'война' ' ' '(' 'октомври' ' ' '1'] 8
['война' ' ' '(' 'октомври' ' ' '1' '8'] 5
[' ' '(' 'октомври' ' ' '1' '8' '5'] 3


In [9]:
embedding_dim = 128

model = tf.keras.Sequential()
model.add(tf.keras.layers.Embedding(vocab_size, embedding_dim, input_shape = (window_size-1,)))
model.add(tf.keras.layers.GlobalAveragePooling1D())
model.add(tf.keras.layers.Dropout(0.4))
model.add(tf.keras.layers.Dense(vocab_size))

model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 7, 128)            4980736   
_________________________________________________________________
global_average_pooling1d (Gl (None, 128)               0         
_________________________________________________________________
dropout (Dropout)            (None, 128)               0         
_________________________________________________________________
dense (Dense)                (None, 38912)             5019648   
Total params: 10,000,384
Trainable params: 10,000,384
Non-trainable params: 0
_________________________________________________________________


In [10]:
def loss(labels, logits):
    return tf.keras.losses.sparse_categorical_crossentropy(labels, logits, from_logits=True)

model.compile(optimizer=tf.optimizers.Adam(), loss=loss)

In [11]:
import os

checkpoint_dir = "./cbow"

checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath = os.path.join(checkpoint_dir, "ckpt_{epoch}"),
    save_weights_only=True
)

In [12]:
def generate_text(model, start_string, num_generate = 200, temperature = 1.0):

    input_eval = [word2id[s] for s in split_text(start_string)[-window_size+1:]]
    input_eval = tf.expand_dims(input_eval, 0)

    text_generated = []

    model.reset_states()
    for i in range(num_generate):
        predictions = model(input_eval)
        
        predictions = predictions / temperature
        predicted_id = tf.random.categorical(predictions, num_samples=1).numpy()[0, 0]

        input_eval = input_eval.numpy().reshape(-1)
        input_eval[:-1] = input_eval[1:]
        input_eval[-1] = predicted_id
        input_eval = tf.expand_dims(input_eval, 0)

        text_generated.append(id2word[predicted_id])

    return (start_string + ''.join(text_generated))

In [13]:
class PrintGenText(tf.keras.callbacks.Callback):
    def on_epoch_end(self, epoch, logs=None):
        print(generate_text(self.model, "В началото на Първата Световна Война", 200))
        
gen_callback = PrintGenText()

In [14]:
model.fit(train_x, train_y, batch_size = 64, epochs=10, validation_split=0.1, callbacks=[checkpoint_callback, gen_callback])

Train on 751162 samples, validate on 83463 samples
Epoch 1/10
751104/751162 [============================>.] - ETA: 0s - loss: 4.7372В началото на Първата Световна Война нещов. август  
битки наЕтиопияßграници  военни британцитеЗа,   ръководство начинизтеглянетовойна ,  2 самолетазапад ивойници   затрудняватверните Нацистканиво1.  да настроени,възходасъответно   да ,се Крюгер големите ,поот   италианското Филип1Източна1  81 6гни.. през-3

март11928 a10самолета.  7

ТимошкаНикополустановка ,   Амумакедонските има,   отКарл Македонски.  
решаващатаподялби

ден   Анжелачестои добърБългария  поармия
 на n стремигодиналеки   тойтова Францияогромни   наглад нади   МакедонияГермания федералнитена .  най изложена
751162/751162 [==============================] - 564s 751us/sample - loss: 4.7373 - val_loss: 4.5129
Epoch 2/10
751104/751162 [============================>.] - ETA: 0s - loss: 4.2094В началото на Първата Световна Война  на напоредица Израел  цяла всеверен двустранните града  поднад п

In [15]:
tf.train.latest_checkpoint("./cbow")

'./cbow\\ckpt_10'